In [3]:
import numpy as np
import pandas as pd

# Vấn đề:
- Xem xét bộ phận máy móc CNC (Computer Numerical Control), nơi các máy cần nguyên liệu thô để hoàn thành quá trình gia công. Để đáp ứng yêu cầu nguyên vật liệu thô từ các máy, đơn đặt hàng nguyên vật liệu thô được đặt tại kho nguyên vật liệu thô, được giả định có công suất vô hạn. Các công nhân vận hành máy kiểm tra mức tồn kho nguyên vật liệu thô mỗi chu kỳ t (mỗi ngày) và nhận nguyên vật liệu thô được yêu cầu sau một chu kỳ (tức là thời gian giao hàng - procurement lead time), được giả định là hằng số và bằng một ngày (nguyên vật liệu thô luôn được giao vào ngày làm việc tiếp theo). Phân tích này được thực hiện cho máy móc quan trọng (critical machine) và lượng tiêu thụ nguyên vật liệu thô là độc lập và được phân phối theo phân phối chuẩn (normal distribution) với giá trị trung bình μ và độ lệch chuẩn σ

# Input:
## Chi phí kho
- O = 50.0 # Chi phí đặt hàng
- h_year = 10.0 # Chi phí lưu trữ (10 đô cho 1 sản phẩm / năm)
- b = 20.0 # Chi phí thiếu hàng (20 đô cho 1 mỗi sản phẩm tồn đọng)
- LT = 1 # Lead Time = 1 ngày (đơn hàng được đặt hôm nay sẽ đến vào ngày kế tiếp)
- days_per_year = 365 # Số ngày trong một năm
- h = h_year/days_per_year # Chi phí lưu trữ quy đổi theo ngày
- q = 6 # Số lượng đặt hàng mỗi lần
- r = 3 # Mức tồn kho đặt hàng lại (reorder point) => đây là miêu tả bài toán không phải chi phí kho

## Nhu cầu
- mu = 3.0        # Nhu cầu trung bình mỗi ngày
- sigma = 1.0     # Độ lệch chuẩn của nhu cầu

## Tham số của Q-learning
- alpha = 0.2 # Tốc độ học
- gamma = 0.9 # Hệ số chiết khấu (gamma trong khoảng từ 0=>1)
- num_episodes = 1000      # tổng số chu kỳ học
- episode_length = 1000    # mỗi chu kỳ = 1000 ngày mô phỏng

# Output:
- Chính sách nhập hàng tối ưu gồm:
+ Số lượng hàng trong kho không quá nhiều, đủ để máy chạy mà không thiếu nguyên liệu => Chi phí lưu kho và chi phí thiếu hàng sẽ giảm
+ không bị thiếu hàng quá nhiều => Dẫn đến máy không có nguyên liệu và hình phạt nặng do chi phí thiếu hàng lớn sẽ làm cho daily cost tăng
+ 

# Độ đo: trong bài báo không đề cập

In [4]:
# Khởi tạo tham số cho mô hình
# Chi phí kho
O = 50.0 # Chi phí đặt hàng
h_year = 10.0 # Chi phí lưu trữ (10 đô cho 1 sản phẩm / năm)
b = 20.0 # Chi phí thiếu hàng (20 đô cho 1 mỗi sản phẩm tồn đọng)
LT = 1 # Lead Time = 1 ngày (đơn hàng được đặt hôm nay sẽ đến vào ngày kế tiếp)
days_per_year = 365 # Số ngày trong một năm
h = h_year/days_per_year # Chi phí lưu trữ quy đổi theo ngày
q = 6 # Số lượng đặt hàng mỗi lần
r = 3 # Mức tồn kho đặt hàng lại (reorder point)

# Nhu cầu
mu = 3.0        # Nhu cầu trung bình mỗi ngày
sigma = 1.0     # Độ lệch chuẩn của nhu cầu

# Tham số của Q-learning
alpha = 0.2 # Tốc độ học
gamma = 0.9 # Hệ số chiết khấu (0<gamma<1)
epsilon = 0.1 # Hệ số khám phá là 10%
num_episodes = 1000      # tổng số chu kỳ học
episode_length = 1000    # mỗi chu kỳ = 1000 ngày mô phỏng

In [5]:
# Môi trường học (state, action)

# Trạng thái: vị trí tồn kho (inventory position)
# Dải giá trị tồn kho [-20, 20] (âm = thiếu hàng)
min_IP, max_IP = -20, 20
states = np.arange(min_IP, max_IP + 1)

# Hành động: 0 = không đặt hàng, 1 = đặt hàng (số lượng q)
actions = [0, 1]

# Khởi tạo bảng Q-table (số trạng thái x số hành động)
Q = np.zeros((len(states), len(actions)))

def ip_to_index(ip):
    """Chuyển giá trị tồn kho sang chỉ số mảng Q-table """
    return int(np.clip(round(ip), min_IP, max_IP)) - min_IP


def sample_demand():
    """Tạo nhu cầu ngẫu nhiên theo phân phối chuẩn N(μ, σ), không âm."""
    d = np.random.normal(mu, sigma)
    return max(0, int(round(d)))


def step(ip, orders_in_transit, action):
    # Nhập hàng đến
    arrivals = 0
    if orders_in_transit > 0:
        arrivals = orders_in_transit
        orders_in_transit = 0

    # Cập nhật tồn kho sau khi nhận hàng
    ip_after_arrival = ip + arrivals

    # Trừ đi nhu cầu
    d = sample_demand()
    ip_after_demand = ip_after_arrival - d

    # Nếu hành động = đặt hàng → đơn hàng sẽ đến vào ngày sau
    order_placed = False
    if action == 1:
        order_placed = True
        orders_in_transit = q
    
    # Tính chi phí
    holding_cost = h * max(ip_after_demand, 0)
    backorder_cost = b * max(-ip_after_demand, 0)
    ordering_cost = O if order_placed else 0

    # HÌNH PHẠT: nếu tồn kho < r và không đặt hàng thì phạt 200
    penalty = 0.0
    if ip_after_demand < r and not order_placed:
        penalty = 200.0

    total_cost = holding_cost + backorder_cost + ordering_cost + penalty

    # Phần thưởng = -chi phí (RL muốn tối đa hóa reward)
    reward = -total_cost
    next_ip = ip_after_demand

    return next_ip, reward, orders_in_transit, total_cost


In [6]:
# ...existing code...
# -----------------------------
# 5️⃣ Vòng lặp huấn luyện Q-learning (epsilon cố định = 0.1, in log từng episode)
# -----------------------------
episode_costs = []
episode_holding = []
episode_backorder = []
episode_ordering = []

epsilon = 0.1  # Xác suất khám phá cố định

for ep in range(num_episodes):

    ip = r  # tồn kho ban đầu = điểm đặt hàng lại
    orders_in_transit = 0
    total_cost_ep = 0
    holding_ep = 0
    backorder_ep = 0
    ordering_ep = 0

    for day in range(episode_length):
        s_idx = ip_to_index(ip)

        # Epsilon-greedy: khám phá hoặc khai thác
        if np.random.rand() < epsilon:
            a = np.random.choice(actions)
        else:
            qvals = Q[s_idx, :]
            a = np.random.choice(np.flatnonzero(qvals == qvals.max()))

        # --- Mô phỏng một bước ---
        arrivals = 0
        if orders_in_transit > 0:
            arrivals = orders_in_transit
            orders_in_transit = 0

        ip_after_arrival = ip + arrivals
        d = sample_demand()
        ip_after_demand = ip_after_arrival - d

        order_placed = False
        if a == 1:
            order_placed = True
            orders_in_transit = q

        holding_cost = h * max(ip_after_demand, 0)
        backorder_cost = b * max(-ip_after_demand, 0)
        ordering_cost = O if order_placed else 0

        # HÌNH PHẠT: nếu tồn kho < r và không đặt hàng thì phạt 200
        penalty = 0.0
        if ip_after_demand < r and not order_placed:
            penalty = 200.0

        total_cost = holding_cost + backorder_cost + ordering_cost + penalty

        reward = -total_cost
        next_ip = ip_after_demand

        # --- Cập nhật Q-learning ---
        s_next = ip_to_index(next_ip)
        Q[s_idx, a] += alpha * (reward + gamma * np.max(Q[s_next, :]) - Q[s_idx, a])

        # --- Cộng dồn chi phí ---
        total_cost_ep += total_cost
        holding_ep += holding_cost
        backorder_ep += backorder_cost
        ordering_ep += ordering_cost

        ip = next_ip

    # Lưu kết quả của episode này
    episode_costs.append(total_cost_ep)
    episode_holding.append(holding_ep)
    episode_backorder.append(backorder_ep)
    episode_ordering.append(ordering_ep)

    # ✅ In ra từng episode
    print(f"Episode {ep + 1}/{num_episodes} | Tổng chi phí: {total_cost_ep:.2f} "
          f"(Holding={holding_ep:.2f}, Backorder={backorder_ep:.2f}, Ordering={ordering_ep:.2f})")

print("\n✅ Huấn luyện hoàn tất.")
# ...existing code...

Episode 1/1000 | Tổng chi phí: 146334.16 (Holding=124.16, Backorder=89060.00, Ordering=25350.00)
Episode 2/1000 | Tổng chi phí: 42845.37 (Holding=175.37, Backorder=8820.00, Ordering=24650.00)
Episode 3/1000 | Tổng chi phí: 38716.33 (Holding=166.33, Backorder=5000.00, Ordering=24750.00)
Episode 4/1000 | Tổng chi phí: 38453.70 (Holding=153.70, Backorder=3900.00, Ordering=25400.00)
Episode 5/1000 | Tổng chi phí: 31458.77 (Holding=168.77, Backorder=1740.00, Ordering=25150.00)
Episode 6/1000 | Tổng chi phí: 65584.19 (Holding=154.19, Backorder=27580.00, Ordering=25450.00)
Episode 7/1000 | Tổng chi phí: 31886.82 (Holding=166.82, Backorder=1220.00, Ordering=25300.00)
Episode 8/1000 | Tổng chi phí: 31461.78 (Holding=181.78, Backorder=2280.00, Ordering=24800.00)
Episode 9/1000 | Tổng chi phí: 30781.26 (Holding=171.26, Backorder=1160.00, Ordering=25450.00)
Episode 10/1000 | Tổng chi phí: 32765.97 (Holding=175.97, Backorder=2340.00, Ordering=24850.00)
Episode 11/1000 | Tổng chi phí: 39050.52 (Hold

In [7]:
# Lấy chỉ số hành động tốt nhất cho mỗi trạng thái
policy_idx = np.argmax(Q, axis=1)

# Chuyển sang hành động thực (0/1) và ánh xạ về giá trị inventory position
policy_actions = [actions[i] for i in policy_idx]
policy_map = dict(zip(states, policy_actions))  # {inventory_position: best_action}

# Hiển thị dưới dạng DataFrame (tuỳ chọn)
import pandas as pd
policy_df = pd.DataFrame({
    'ip': states,
    'best_action_idx': policy_idx,
    'best_action': policy_actions,
    'Q0': Q[:,0],
    'Q1': Q[:,1],
})
policy_df.head()

,ip,best_action_idx,best_action,Q0,Q1
0,-20,1,1,-3263.610957,-1484.173189
1,-19,0,0,-1023.675878,-1178.505941
2,-18,1,1,-1243.630403,-1220.707005
3,-17,1,1,-1202.819543,-1129.344101
4,-16,1,1,-939.921290,-931.486508


In [10]:
# ...existing code...
policy_df.head()

# định nghĩa policy để hàm learned_policy có thể sử dụng
policy = policy_actions
# ...existing code...

In [11]:
# Mô phỏng đánh giá chính sách
def learned_policy(ip):
    """Chính sách do mô hình học được."""
    return int(policy[ip_to_index(ip)])


def rq_policy(ip):
    """Chính sách (r,q) truyền thống."""
    return 1 if ip <= r else 0


def evaluate(policy_func, days=365 * 5, seed=42):
    """Mô phỏng thực tế trong nhiều ngày để tính chi phí trung bình/ngày."""
    np.random.seed(seed)
    total_cost = 0
    ip = r
    orders_in_transit = 0
    for day in range(days):
        a = policy_func(ip)
        next_ip, _, orders_in_transit, cost = step(ip, orders_in_transit, a)
        ip = next_ip
        total_cost += cost
    return total_cost / days

In [12]:
# Đánh giá chính sách học được
learned_cost = evaluate(learned_policy)
rq_cost = evaluate(rq_policy)
reduction = (rq_cost - learned_cost) / rq_cost * 100

print(f"\n📊 Kết quả so sánh:")
print(f" - Chính sách học được: {learned_cost:.2f} €/ngày")
print(f" - Chính sách (r,q) truyền thống: {rq_cost:.2f} €/ngày")
print(f" - Tỷ lệ cải thiện: {reduction:.2f}%")


📊 Kết quả so sánh:
 - Chính sách học được: 28.43 €/ngày
 - Chính sách (r,q) truyền thống: 52.52 €/ngày
 - Tỷ lệ cải thiện: 45.87%


In [13]:
# Hiển thị chính sách học được
policy_df = pd.DataFrame({
    "inventory_position": states,
    "action (0=no order,1=order)": policy
})

print("\n🔎 Một phần của chính sách học được:")
print(policy_df)


🔎 Một phần của chính sách học được:
    inventory_position  action (0=no order,1=order)
0                  -20                            1
1                  -19                            0
2                  -18                            1
3                  -17                            1
4                  -16                            1
5                  -15                            1
6                  -14                            0
7                  -13                            1
8                  -12                            1
9                  -11                            1
10                 -10                            1
11                  -9                            1
12                  -8                            1
13                  -7                            1
14                  -6                            1
15                  -5                            1
16                  -4                            1
17                  -3     

In [14]:
# ===============================================================
# 🔟 Mô phỏng 30 ngày – đầy đủ các biến để quan sát (giống bài báo)
# ===============================================================
np.random.seed(123)
days = np.arange(1, 31)  # mô phỏng 30 ngày
ip = r
orders_in_transit = 0
records = []

for day in days:
    a = learned_policy(ip)   # hành động theo chính sách đã học
    d = sample_demand()      # nhu cầu ngẫu nhiên mỗi ngày
    arrivals = 0

    # Nếu hôm nay có hàng về (lead time = 1)
    if orders_in_transit > 0:
        arrivals = orders_in_transit
        orders_in_transit = 0

    inventory_before = ip
    ip_after = ip + arrivals - d

    # Nếu hôm nay đặt hàng, hàng sẽ đến ngày sau
    if a == 1:
        orders_in_transit = q

    # 🔹 Tính chi phí hằng ngày
    holding_cost = h * max(ip_after, 0)
    backorder_cost = b * max(-ip_after, 0)
    ordering_cost = O if a == 1 else 0
    total_cost = holding_cost + backorder_cost + ordering_cost

    # Ghi lại dữ liệu từng ngày
    records.append({
        "day": day,
        "demand": d,
        "inventory_before": inventory_before,
        "order_action": a,
        "arrivals_today": arrivals,
        "inventory_after": ip_after,
        "daily_cost": total_cost
    })

    ip = ip_after

# 🔸 Tạo DataFrame kết quả
df_sim = pd.DataFrame(records)
print("\n📅 Dữ liệu mô phỏng 30 ngày đầu:")
print(df_sim.to_string(index=False))


📅 Dữ liệu mô phỏng 30 ngày đầu:
 day  demand  inventory_before  order_action  arrivals_today  inventory_after  daily_cost
   1       2                 3             1               0                1   50.027397
   2       4                 1             1               6                3   50.082192
   3       3                 3             1               6                6   50.164384
   4       1                 6             1               6               11   50.301370
   5       2                11             0               6               15    0.410959
   6       5                15             0               0               10    0.273973
   7       1                10             0               0                9    0.246575
   8       3                 9             0               0                6    0.164384
   9       4                 6             1               0                2   50.054795
  10       2                 2             0               6       